In [5]:
import pandas as pd
import time
import os
import json
import io
from collections import Counter
from os.path import join,isfile
from os import listdir
import dask
from fake_useragent import UserAgent
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
import bookquery
ua = UserAgent()#Прокси

def counter_for_files(path):# Подсчет файлов в директории
    onlyfiles = [join(path, f) for f in listdir(path) if isfile(join(path, f))]
    return onlyfiles

In [6]:
def execute_query(qquery, entity_id):# Работа с запросами
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql",agent = ua.random)
    sparql.setQuery(qquery.format(entity_id = entity_id))
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return results['results']['bindings']

In [7]:
not_found = []

def find_entity(entity):
    path = join(os.getcwd() , r"Information\2\data\rus_labels")
    res = []
    dic = {}
    for top, dirs, files in os.walk(path):
        for f in files:
            file_path = join(path,f)
            exl = pd.read_csv(io.open(file_path, encoding='utf-8'), sep = ';', header = 0)
            entity_list = exl['entity'].tolist()
            if entity in entity_list:
                return f.split('_')[-1].split('.')[0], f.split('_')[1]
    global not_found
    not_found.append(entity)
    return None

In [ ]:
%%time
all_files = []
ones = ["count_constraint","max_by_area_constraint","max_by_height_constraint","max_by_length_constraint",
"wash"]
path = join(os.getcwd() , f'verified_files')
for top, dirs, files in os.walk(path): #проходимся по всем екселям (интентам)
    for f in files:
        count_x = 0
        intent_name = f.split('.')[0] #имя интента|имя файла
        exl = pd.read_excel(os.path.join(top, f)) #ексель блохина
        if intent_name not in ones:
            old_qid = exl[exl['Сущность #0 найдена правильно'] == 1]['QID #0'] #берем нужный столбец, неисправленное QID
            new_qid = exl[exl['Сущность #0 найдена правильно'] == 0]['Правильная сущность #0'] #исправленные QID
            print(intent_name + '_0')
        else:
            old_qid = exl[exl['Сущность #1 найдена правильно'] == 1]['QID #1'] #берем нужный столбец, неисправленное QID
            new_qid = exl[exl['Сущность #1 найдена правильно'] == 0]['Правильная сущность #1'] #исправленные QID
            print(intent_name + '_1')
        qid = old_qid.tolist() + new_qid.tolist()
        res = []
        result = []
        exl_type = pd.read_excel(join(os.getcwd(),f'csv\Entities.xlsx')) #проверяем является ли сущность типом
        qid_type = exl_type[exl_type['subclass_weight'] == 2]['subclass']
        for i in qid:
            if i != 'x' and i != 'х':
                if i in qid_type.tolist(): #проверяем является ли сущность типом если да, сразу в res
                    type_lable = exl_type[exl_type['subclass'] == i]['subclass_label'].values[0]
                    a = (i, type_lable)
                    result.append(a)
                else:
                    result.append(find_entity(i))
            else:
                count_x += 1
        c = Counter()
        for i in result:
            c[i] += 1 
        all_files.append((c,intent_name, len(exl),count_x))

In [10]:
%%time
all_files = []
ones = ["count_constraint","max_by_area_constraint","max_by_height_constraint","max_by_length_constraint",
"wash"]
path = join(os.getcwd() , f'verified_files_1')
for top, dirs, files in os.walk(path): #проходимся по всем екселям (интентам)
    for f in files:
        count_x = 0
        intent_name = f.split('.')[0] #имя интента|имя файла
        exl = pd.read_excel(os.path.join(top, f)) #ексель блохина
        if intent_name in ones:
            old_qid = exl[exl['Сущность #0 найдена правильно'] == 1]['QID #0'] #берем нужный столбец, неисправленное QID
            new_qid = exl[exl['Сущность #0 найдена правильно'] == 0]['Правильная сущность #0'] #исправленные QID
            print(0)
        else:
            old_qid = exl[exl['Сущность #1 найдена правильно'] == 1]['QID #1'] #берем нужный столбец, неисправленное QID
            new_qid = exl[exl['Сущность #1 найдена правильно'] == 0]['Правильная сущность #1'] #исправленные QID
            print(1)
        qid = old_qid.tolist() + new_qid.tolist()
        res = []
        result = []
        exl_type = pd.read_excel(join(os.getcwd(),f'csv\Entities.xlsx')) #проверяем является ли сущность типом
        qid_type = exl_type[exl_type['subclass_weight'] == 2]['subclass']
        for i in qid:
            if i != 'x' and i != 'х':
                if i in qid_type.tolist(): #проверяем является ли сущность типом если да, сразу в res
                    type_lable = exl_type[exl_type['subclass'] == i]['subclass_label'].values[0]
                    a = (i, type_lable)
                    result.append(a)
                else:
                    result.append(find_entity(i))
            else:
                count_x += 1
        c = Counter()
        for i in result:
            c[i] += 1 
        all_files.append((c,intent_name, len(exl),count_x))

0
1
0
0
0
1
0
Wall time: 2min 5s


In [11]:
# {item[1] : item[3] for item in all_files}
# all_files
[item[2] for item in all_files]

[306, 363, 358, 290, 261, 305, 349]

In [ ]:
name_dict = {"area" : "площадь(P2046)",
            "borders_country" : "имеет границы с(P47)",
             "borders_country_count" : "имеет границы с(P47)",
            "capital" : "столица(P36)",
             "climate" : "климат(P2564)",
             "count": "just_labels", 
             "count_constraint": "just_labels",
            "country_by_capital" : "является столицей(P1376)",
             "depth" : "глубина(P4511)",
             "height":"высота над уровнем моря(P2044)",
             "length" : "длина(P2043)",
             "list_rivers":"just_labels",
             "location" : "just_labels",
             "location_constraint" : "just_labels",
             "location_coordinates" : "координаты(P625)",
             "max_by_area":"площадь(P2046)",
             "max_by_area_constraint" : "площадь(P2046)",
             "max_by_depth":"глубина(P4511)",
             "max_by_height":"высота над уровнем моря(P2044)",
             "max_by_height_constraint" : "высота над уровнем моря(P2044)",
             "max_by_length" : "длина(P2043)",
             "max_by_length_constraint" : "длина(P2043)",
             "max_by_population_count" : "население(P1082)",
             "min_by_area" : "площадь(P2046)",
             "political_own" : "государство(P17)",
            "population_count" : "население(P1082)",
            "river_confluence" : "впадает в(P403)" ,
            "river_inflows": "приток(P974)",
            "sight_location": "just_labels" ,
            "wash": "на берегу(P206)"}
exl_file = pd.ExcelFile(join(os.getcwd() , f'Final analytics\\Analytics.xlsx'))
df = exl_file.parse(name_dict[all_files[0][1]])
df['global'] = 0
count_rows = 0
count_xx = 0
for items in all_files:
    del items[0][None]
#     print(items)
    qid_list = list(map(lambda x :x[0] , list(items[0].keys())))
    for k,v in df.iterrows():
        ind = list(df[df.subclass == v.subclass].index)[0]
        if v.subclass in qid_list:
            df.loc[ind,items[1]] = round(items[0][(v.subclass , v.subclass_label)]/items[2] * 100,2)
        else:
            df.loc[ind,items[1]] = "-"
new_row = {'subclass_label' : 'x' , 'subclass' : 'x'}
new_row.update({item[1] : round(item[3] * 100 /item[2],2) for item in all_files})
df = df.append(pd.DataFrame.from_dict([new_row], orient='columns', dtype=None, columns=None))
lst = ["subclass_label" , "subclass"]
lst.extend(list(name_dict.keys()))
df[lst].to_excel(r'Final analytics\new_Analytics.xlsx', sheet_name = 'analysis_by_intents' ,encoding='utf-8-sig' , index = False)

df = exl_file.parse(name_dict[all_files[0][1]])
df['global'] = 0
for items in all_files:
    count_rows += items[2]
    count_xx+= items[3]
#     print(items)
    qid_list = list(map(lambda x :x[0] , list(items[0].keys())))
    for k,v in df.iterrows():
        ind = list(df[df.subclass == v.subclass].index)[0]
        if v.subclass in qid_list:
            df.loc[ind,'global'] += items[0][(v.subclass , v.subclass_label)]
        else:
            df.loc[ind,'global'] += 0
df['global'] = round((df['global']/count_rows) * 100,2)
for k,v in df.iterrows():
    if v['global'] == 0:
        ind = list(df[df.subclass == v.subclass].index)[0]
        df.loc[ind,'global'] = '-'
print(count_rows)
print(count_xx)
new_row = {'subclass_label' : 'x' , 'subclass' : 'x', 'global' :round((count_xx/count_rows) * 100,2)}
df = df.append(pd.DataFrame.from_dict([new_row], orient='columns', dtype=None, columns=None))
lst = ["subclass_label" , "subclass" , "global"]
with pd.ExcelWriter(r'Final analytics\new_Analytics.xlsx' ,  engine="openpyxl" , mode  = 'a') as writer:
    df[lst].to_excel(writer, sheet_name = 'global_analysis' , encoding='utf-8-sig' , index = False)   
    
for items in all_files:
    del items[0][None]
    df = exl_file.parse(name_dict[items[1]])
#     print(items)
    qid_list = list(map(lambda x :x[0] , list(items[0].keys())))
    for k,v in df.iterrows():
        ind = list(df[df.subclass == v.subclass].index)[0]
        if v.subclass in qid_list:
            df.loc[ind,'Абсолютное значение'] = items[0][(v.subclass , v.subclass_label)]
            df.loc[ind,'Относительное значение , %'] = round(items[0][(v.subclass , v.subclass_label)]/items[2] * 100,2)
        else:
            df.loc[ind,'Абсолютное значение'] = '-'
            df.loc[ind,'Относительное значение , %'] = '-'
    new_row = {'subclass_label' : 'x' , 'subclass' : 'x', 'Встречаемость предиката для всех (%)' : '-' ,
               'Встречаемость предиката для популярных(%)': '-' , 'Абсолютное значение' : items[3] ,
               'Относительное значение , %' : round(items[3]* 100/items[2] , 2) }
    df = df.append(pd.DataFrame.from_dict([new_row], orient='columns', dtype=None, columns=None))
    for k,v in df.iterrows():
        ind = list(df[df.subclass == v.subclass].index)[0]
        if v['Абсолютное значение'] == 0:
            df.loc[ind,'Абсолютное значение'] = '-'
        if v['Относительное значение , %'] == 0:
            df.loc[ind,'Относительное значение , %'] = '-'
    with pd.ExcelWriter(r'Final analytics\new_Analytics.xlsx' ,  engine="openpyxl" , mode  = 'a') as writer:
        df.to_excel(writer, sheet_name = items[1] , encoding='utf-8-sig' , index = False)

In [12]:
name_dict = {             "count_constraint": "just_labels",
             "location_constraint" : "just_labels",
             "max_by_area_constraint" : "площадь(P2046)",
             "max_by_height_constraint" : "высота над уровнем моря(P2044)",
             "max_by_length_constraint" : "длина(P2043)",
            "sight_location": "just_labels" ,
            "wash": "на берегу(P206)"}
exl_file = pd.ExcelFile(join(os.getcwd() , f'Final analytics\\Analytics.xlsx'))
df = exl_file.parse(name_dict[all_files[0][1]])
df['global'] = 0
count_rows = 0
count_xx = 0
for items in all_files:
    del items[0][None]
#     print(items)
    qid_list = list(map(lambda x :x[0] , list(items[0].keys())))
    for k,v in df.iterrows():
        ind = list(df[df.subclass == v.subclass].index)[0]
        if v.subclass in qid_list:
            df.loc[ind,items[1]] = round(items[0][(v.subclass , v.subclass_label)]/items[2] * 100,2)
        else:
            df.loc[ind,items[1]] = "-"
new_row = {'subclass_label' : 'x' , 'subclass' : 'x'}
new_row.update({item[1] : round(item[3] * 100 /item[2],2) for item in all_files})
df = df.append(pd.DataFrame.from_dict([new_row], orient='columns', dtype=None, columns=None))
lst = ["subclass_label" , "subclass"]
lst.extend(list(name_dict.keys()))
df[lst].to_excel(r'Final analytics\new_Analytics_1.xlsx', sheet_name = 'analysis_by_intents' ,encoding='utf-8-sig' , index = False)

df = exl_file.parse(name_dict[all_files[0][1]])
df['global'] = 0
for items in all_files:
    count_rows += items[2]
    count_xx+= items[3]
#     print(items)
    qid_list = list(map(lambda x :x[0] , list(items[0].keys())))
    for k,v in df.iterrows():
        ind = list(df[df.subclass == v.subclass].index)[0]
        if v.subclass in qid_list:
            df.loc[ind,'global'] += items[0][(v.subclass , v.subclass_label)]
        else:
            df.loc[ind,'global'] += 0
df['global'] = round((df['global']/count_rows) * 100,2)
for k,v in df.iterrows():
    if v['global'] == 0:
        ind = list(df[df.subclass == v.subclass].index)[0]
        df.loc[ind,'global'] = '-'
print(count_rows)
print(count_xx)
new_row = {'subclass_label' : 'x' , 'subclass' : 'x', 'global' :round((count_xx/count_rows) * 100,2)}
df = df.append(pd.DataFrame.from_dict([new_row], orient='columns', dtype=None, columns=None))
lst = ["subclass_label" , "subclass" , "global"]
with pd.ExcelWriter(r'Final analytics\new_Analytics_1.xlsx' ,  engine="openpyxl" , mode  = 'a') as writer:
    df[lst].to_excel(writer, sheet_name = 'global_analysis' , encoding='utf-8-sig' , index = False)   
    
for items in all_files:
    del items[0][None]
    df = exl_file.parse(name_dict[items[1]])
#     print(items)
    qid_list = list(map(lambda x :x[0] , list(items[0].keys())))
    for k,v in df.iterrows():
        ind = list(df[df.subclass == v.subclass].index)[0]
        if v.subclass in qid_list:
            df.loc[ind,'Абсолютное значение'] = items[0][(v.subclass , v.subclass_label)]
            df.loc[ind,'Относительное значение , %'] = round(items[0][(v.subclass , v.subclass_label)]/items[2] * 100,2)
        else:
            df.loc[ind,'Абсолютное значение'] = '-'
            df.loc[ind,'Относительное значение , %'] = '-'
    new_row = {'subclass_label' : 'x' , 'subclass' : 'x', 'Встречаемость предиката для всех (%)' : '-' ,
               'Встречаемость предиката для популярных(%)': '-' , 'Абсолютное значение' : items[3] ,
               'Относительное значение , %' : round(items[3]* 100/items[2] , 2) }
    df = df.append(pd.DataFrame.from_dict([new_row], orient='columns', dtype=None, columns=None))
    for k,v in df.iterrows():
        ind = list(df[df.subclass == v.subclass].index)[0]
        if v['Абсолютное значение'] == 0:
            df.loc[ind,'Абсолютное значение'] = '-'
        if v['Относительное значение , %'] == 0:
            df.loc[ind,'Относительное значение , %'] = '-'
    with pd.ExcelWriter(r'Final analytics\new_Analytics_1.xlsx' ,  engine="openpyxl" , mode  = 'a') as writer:
        df.to_excel(writer, sheet_name = items[1] , encoding='utf-8-sig' , index = False)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


2232
39


In [13]:
ua = UserAgent()
res = []
bad_requests = []

def execute_query(qquery, entity_id):
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql", agent = ua.random)
    sparql.setQuery(qquery.format(entity_id = entity_id))
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return results['results']['bindings']

def entities_sparqul(entity, f = 0):
        entities = {}
        res = []
        s = ''
        if f > 3:
            global bad_requests
            bad_requests.append(entity)
            print('Bad request ', entity)
            return None
        try:
            raw_results = execute_query(bookquery.subclass_of, entity)
            if raw_results != []:
                entities['entity_id'] = raw_results[0]['show']['value'].split('/')[-1]
                entities['entity_label'] = raw_results[0]['showLabel']['value']
                for r in raw_results:
                    subclass = r['subclass']['value'].split('/')[-1]
                    subclass_label = r['subclassLabel']['value']
                    s += subclass + '_' + subclass_label + ','
                entities['subclass'] = s
            else:
                raw_results = execute_query(bookquery.subclass_name, entity)
                
                entities['entity_id'] = raw_results[0]['show']['value'].split('/')[-1]
                entities['entity_label'] = raw_results[0]['showLabel']['value']
                entities['subclass'] = s
#             print(entities)
            res.append(entities)
            return entities #pd.DataFrame(data = res, columns = ['entity_id', 'entity_label', 'subclass'])
        except Exception as e:
            time.sleep(5)
            print(f'Exception in {entity}: {e}')
            return entities_sparqul(entity, f + 1)

In [14]:
print(len(not_found))

251


In [15]:
%%time
res = []
for i in not_found:
    if i != 'x' or i != 'х':
        res.append(entities_sparqul(i))

Wall time: 2min 3s


In [16]:
f = []
a = []
c = Counter()
for i in not_found:
    c[i] += 1 
count = dict(c)
for i in res:
    if i['entity_id'] in count and not i['entity_id'] in a:
        i['count'] = count[i['entity_id']]
        f.append(i)
        a.append(i['entity_id'])
f = pd.DataFrame(data = f)
f = f[['entity_id','entity_label','count','subclass']].sort_values(by = 'count',ascending = False)
f.to_excel(r'Final analytics\Other_entities_1.xlsx', sheet_name = 'by_entities' ,encoding='utf-8-sig' , index = False)


In [17]:
b = []
for i in res:
    if i['subclass'] != '':
        a = i['subclass'].split(",")
        for j in a:
            if j != '' and not 'известная по народному названию' in j:
                qid = j.split('_')[0]
                label = j.split('_')[1]
                b.append((qid, label))
c = Counter()
for i in b:
    c[i] += 1 
result = []
for k,v in c.items():
    dic = {}
    dic['entity_id'] = k[0]
    dic['subclass'] = k[1]
    dic['count'] = v
    result.append(dic)
df = pd.DataFrame(data = result).sort_values(by = 'count',ascending = False)
with pd.ExcelWriter(r'Final analytics\Other_entities_1.xlsx' ,  engine="openpyxl" , mode  = 'a') as writer:
        df.to_excel(writer, sheet_name = 'by_subclasses' , encoding='utf-8-sig' , index = False)

In [39]:
ones = ['location_constraint' , 'sight_location']
path = join(os.getcwd() , f'verified_files_1')
result = []
for file in counter_for_files(path):
    df = pd.read_excel(file)
    name = file.split('\\')[-1].split('.')[0]
    c = Counter()
    if  name in ones:
        for k,v in df.iterrows():
            c[(v['Лейбл #1 из Wikidata'] ,v['QID #1'])] += 1 
    else:
        for k,v in df.iterrows():
            c[(v['Лейбл #0 из Wikidata'] ,v['QID #0'])] += 1    
    for k,v in c.items():
        dic = {}
        dic['subclass_label']= k[0]
        dic['subclass'] = k[1]
        dic['Абсолютное значение'] = v
        dic['Относительное значение, %'] = round(v* 100/len(df), 2)
        result.append(dic)
    count_x = 0
    for_del = []
    for i in range(len(result)):
        if result[i]['subclass'] == 'x' or result[i]['subclass'] == 'х':
            count_x+=result[i]['Абсолютное значение']
            for_del.append(i)
    for i in for_del:
        del result[i]
    dic = {'subclass_label' : 'не найдено' , 'subclass' : 'x' , 'Абсолютное значение' : count_x , 'Относительное значение, %' : round(count_x* 100/len(df), 2)}
    df = pd.DataFrame(data = result).sort_values(by = 'Абсолютное значение',ascending = False)
    df = df.append(pd.DataFrame.from_dict([dic], orient='columns', dtype=None, columns=None))
    with pd.ExcelWriter(r'Final analytics\new_Analytics.xlsx' ,  engine="openpyxl" , mode  = 'a') as writer:
         df.to_excel(writer, sheet_name = f'{name}_ss' , encoding='utf-8-sig' , index = False)

In [91]:
exl_file = pd.ExcelFile(join(os.getcwd() , f'Final analytics\\Other_entities.xlsx'))
exl_file_1 = pd.ExcelFile(join(os.getcwd() , f'Final analytics\\Other_entities_1.xlsx'))

df = exl_file.parse('by_entities')
df_1= exl_file_1.parse('by_entities')
new_df = pd.concat([df, df_1])
concat_df = new_df.groupby(['entity_id', 'entity_label']).sum().sort_values(by = 'count',ascending = False)
concat_df.reset_index(inplace = True)
for k,v in concat_df.iterrows():
    ind = list(concat_df[concat_df.entity_id == v.entity_id].index)[0]
    concat_df.loc[ind,'subclass'] = list(new_df[new_df.entity_id == v.entity_id]['subclass'])[0]
concat_df .to_excel(r'Final analytics\Not_found_in_our_pool.xlsx', sheet_name = 'by_entities' ,encoding='utf-8-sig' , index = False )


df = exl_file.parse('by_subclasses')
df_1= exl_file_1.parse('by_subclasses')
new_df = pd.concat([df, df_1])
new_df = new_df.groupby(['entity_id','subclass']).sum().sort_values(by = 'count',ascending = False)
new_df.reset_index(inplace = True)
with pd.ExcelWriter(r'Final analytics\Not_found_in_our_pool.xlsx' ,  engine="openpyxl" , mode  = 'a') as writer:
          new_df.to_excel(writer, sheet_name = f'by_subclasses' , encoding='utf-8-sig' , index = False)